<a href="https://colab.research.google.com/github/deniztahmouresi/interviewTask/blob/develop/InterviewTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import numpy as np
drive.mount('/content/drive')
dataset = np.load('/content/drive/MyDrive/Colab Notebooks/interviewTask/data.npz')


Mounted at /content/drive


In [ ]:
from scipy import signal

x_train = dataset['X_train']
y_train = dataset['y_train']
x_test = dataset['X_test']
y_test = dataset['y_test']

X_train = np.ndarray(shape=(len(x_train),28, 28) , dtype=int)
i = 0
for img in x_train:
  X_train[i] = signal.medfilt2d(img)
  i += 1
X_test = np.ndarray(shape=(len(x_test),28, 28) , dtype=int)
i = 0
for img in x_test:
  X_test[i] = signal.medfilt2d(img)
  i += 1

# Data normalize to the [0, 1] range
X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255

In [ ]:
# Make sure images have shape (28, 28, 1)
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# define data preparation
datagen = ImageDataGenerator(rotation_range = 20)
# configure batch size and retrieve one batch of images
classes = [0, 2, 3, 4, 5, 6, 8, 9]
i=0
x_train_new = np.ndarray(shape=(40296,28, 28, 1), dtype=float)
y_train_new = np.ndarray(shape=(40296,), dtype=int)

for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=1, shuffle=True):
    if(y_batch[0] in classes):
      x_train_new[i] = X_batch
      y_train_new[i] = y_batch
      i += 1
      if i > 40295:
        break
    else:
      continue
print(x_train_new.max())
print(x_train_new.min())
print(x_train_new.shape)
print(y_train_new.shape)
print(x_train_new.max())
print(x_train_new.min())


1.0
0.0
(40296, 28, 28, 1)
(40296,)
1.0
0.0


In [ ]:
classes = [4, 8]
i=0
x_train_new2 = np.ndarray(shape=(10000,28, 28, 1), dtype=float)
y_train_new2 = np.ndarray(shape=(10000,), dtype=int)

for X_batch, y_batch in datagen.flow(X_train, y_train, batch_size=1, shuffle=True):
    if(y_batch[0] in classes):
      x_train_new2[i] = X_batch
      y_train_new2[i] = y_batch
      i += 1
      if i > 9999:
        break
    else:
      continue
print(x_train_new2.max())
print(x_train_new2.min())
print(x_train_new2.shape)
print(y_train_new2.shape)
print(x_train_new2.max())
print(x_train_new2.min())

1.0
0.0
(10000, 28, 28, 1)
(10000,)
1.0
0.0


In [ ]:
X_train = np.concatenate((X_train, x_train_new), axis=0)
#X_train = np.concatenate((X_train, x_train_new2), axis=0)

y_train = np.concatenate((y_train, y_train_new), axis=0)
#y_train = np.concatenate((y_train, y_train_new2), axis=0)

print(X_train.shape)
print(y_train.shape)

(107456, 28, 28, 1)
(107456,)


In [ ]:
count = np.zeros((10,), dtype=int)
print(y_train[0])
for val in y_train:
    val = val.astype(np.int)
    count[val]+=1
print(count*100/sum(count))
print(count)

7.0


<ipython-input-58-65056bf68f66>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  val = val.astype(np.int)


[10.91423466 12.5483919  10.97844699 11.28834127  5.38452948  9.97989875
 10.89283055 11.66058666  5.39569684 10.95704288]
[11728 13484 11797 12130  5786 10724 11705 12530  5798 11774]


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu", kernel_initializer='he_uniform'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu", kernel_initializer='he_uniform'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(100, activation="relu", kernel_initializer='he_uniform'),
        layers.Dense(10, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 1600)              0         
                                                                 
 dense_8 (Dense)             (None, 100)              

In [ ]:
from tensorflow.keras.optimizers import SGD
from sklearn.utils.class_weight import compute_class_weight

batch_size = 128
epochs = 10
class_weights = compute_class_weight(class_weight = "balanced", classes = np.unique(dataset['y_train']), y = dataset['y_train'])
class_weights = [round(w, 2) for w in class_weights]
class_weights = dict(enumerate(class_weights))
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["categorical_accuracy"])
print(X_train.max())
print(X_train.min())
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, class_weight=class_weights)
score = model.evaluate(X_test, y_test, verbose=0)
print(score)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

1.0
0.0
Epoch 1/10
840/840 [==============================] - 83s 98ms/step - loss: 0.1984 - categorical_accuracy: 0.9443
Epoch 2/10
840/840 [==============================] - 82s 97ms/step - loss: 0.0698 - categorical_accuracy: 0.9801
Epoch 3/10
840/840 [==============================] - 81s 96ms/step - loss: 0.0503 - categorical_accuracy: 0.9862
Epoch 4/10
840/840 [==============================] - 81s 96ms/step - loss: 0.0414 - categorical_accuracy: 0.9882
Epoch 5/10
 96/840 [==>...........................] - ETA: 1:14 - loss: 0.0310 - categorical_accuracy: 0.9915